<a href="https://colab.research.google.com/github/amerinor01/Acell-P4/blob/main/Aceleradores_P4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vars

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [8]:
ITER = 10
programs = ('paralelo1.txt','paralelo2.txt', 'paralelo3.txt')
seq_programs = {'secuencial.txt'}
data = []


In [11]:
x = 0
def parserToJson(data):
  result = {
      'Iteration': int(data[0].split(' ')[1]),
      'BlockSize': int(data[0].split(' ')[3]),
      'ArraySize': int(data[0].split(' ')[5]),
      'Stream'   : int(data[0].split(' ')[7]),
      'Kernel Time': float(data[1].split(' ')[2]),
      'Total Time': float(data[2].split(' ')[2]),
      'GFLOPs Time': float(data[3].split(' ')[1]),
  }
  return result

def parserSeqToJson(data):
  result = {
      'Iteration': int(data[0].split(' ')[1]),
      'ArraySize': int(data[0].split(' ')[3]),
      'Time': float(data[1].split(' ')[2]),
  }
  return result;

def getData(filename):
  data = []
  lista = []
  with open(filename) as f:
    content = f.read().splitlines()

  for s in content:
    if s is not '':
      lista.append(s)
    else:
      data.append(parserToJson(lista))
      lista.clear()
  
  return data


def getSeqData(filename):
  data = []
  lista = []
  with open(filename) as f:
    content = f.read().splitlines()
  
  for s in content:
    if s is not '':
      lista.append(s)
    else:
      data.append(parserSeqToJson(lista))
      lista.clear()
  
  return data

def getDataframe(programa):
  x = getData(programa)
  x[0]
  df = pd.DataFrame(x[0],[0])

  for i in range(1,len(x)):
    df = df.append(x[i], ignore_index=True)
    
  #print(x)
  df
  t = df.get(['BlockSize', 'ArraySize','Stream','Kernel Time', 'Total Time', 'GFLOPs Time'])
  p = t.groupby(by=['ArraySize','BlockSize','Stream']).mean()
  return p

def getSeqDataFrame(programa):
  x = getSeqData(programa)
  x[0]
  df = pd.DataFrame(x[0],[0])

  for i in range(1,len(x)):
    df = df.append(x[i], ignore_index=True)
    
  t = df.get(['ArraySize', 'Time'])
  p = t.groupby(by=['ArraySize']).mean()
  

  return p

dfs = []
dfs_seq = []

for x in programs:
  dfs.append(getDataframe(x))

for x in seq_programs:
  dfs_seq.append(getSeqDataFrame(x))


v = 0
with pd.ExcelWriter('output.xlsx') as writer:  
    for df in dfs:
      df.to_excel(writer, sheet_name='Program' + str(v))
      v = v + 1
    for df in dfs_seq:
      df.to_excel(writer, sheet_name='Program' + str(v))
      v = v + 1

